In [1]:
import numpy as np
import librosa
import torch
import laion_clap
from extraction.vgg_sound import *
from audioldm import image_to_audio, build_model, clap_to_audio
import torch
import torchaudio
from transformers import AutoProcessor, AutoModel
import soundfile as sf 
import clip
import torch
from PIL import Image
import torchvision.transforms as transforms

# device = "cuda" if torch.cuda.is_available() else "cpu"
# model, preprocess = clip.load("ViT-B/32", device=device)
# model.eval()




/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
import clip
import torch
from PIL import Image
import torchvision.transforms as transforms

device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()


100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 177MiB/s]


CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [3]:
vgg_dataset = VGGSound(root="/mnt/new_volume/vgg_sound/scratch/shared/beegfs/hchen/train_data/VGGSound_final/video",
                           csv_path="/mnt/new_volume/vgg_sound/test.csv",
                           sample_rate =  16000)


# data = vgg_dataset[7]

INFO:root:199176 videos found in /mnt/new_volume/vgg_sound/scratch/shared/beegfs/hchen/train_data/VGGSound_final/video
INFO:root:15446 videos found in /mnt/new_volume/vgg_sound/test.csv
INFO:root:0 videos missing in /mnt/new_volume/vgg_sound/scratch/shared/beegfs/hchen/train_data/VGGSound_final/video


In [5]:
img = vgg_dataset[0]['clip_video']

sample_rate: 44100
self sample:  16000


In [8]:
img = img.to(device)
# with torch.no_grad():
#     image_embedding = clip_model.encode_image(img) 

In [ ]:
with torch.no_grad():
    image_embedding = clip_model.encode_image(img) 

In [17]:
img[0].shape

torch.Size([3, 224, 224])

In [19]:
with torch.no_grad():
    image_embedding_1 = clip_model.encode_image(img[0].unsqueeze(0).to(device)) 

In [20]:
image_embedding_1

tensor([[ 3.4961e-01,  1.2732e-01, -1.5881e-01,  9.8572e-02,  1.6345e-01,
         -1.9629e-01,  6.2646e-01,  1.9263e-01, -1.3049e-01,  5.4047e-02,
         -4.6906e-02, -1.9800e-01,  1.0040e-01, -4.1724e-01,  3.0151e-01,
         -3.0176e-01, -1.8291e+00, -2.2766e-02,  2.1692e-01, -4.6851e-01,
          2.2827e-01,  1.1505e-01,  1.2891e-01,  2.0276e-01, -6.5079e-03,
         -1.7041e-01, -4.4861e-02, -2.9272e-01,  1.7920e-01,  1.2939e-01,
         -4.6448e-02,  5.4474e-02, -5.4102e-01, -6.2164e-02, -1.0303e-01,
          1.7212e-01, -3.1421e-01,  3.7231e-02,  1.4575e-01, -7.4902e-01,
         -6.3916e-01, -1.5308e-01, -2.6807e-01, -4.7534e-01, -1.4783e-01,
          8.4961e-01,  5.4053e-01, -2.0544e-01, -3.2642e-01,  3.0273e-01,
          5.1514e-01, -3.4399e-01,  1.3745e-01,  2.6758e-01,  2.0874e-01,
          6.0596e-01, -1.9226e-01,  2.3254e-02, -7.5012e-02,  8.9741e-04,
          1.2646e-01, -7.2693e-02,  4.0381e-01,  1.1780e-01,  1.5564e-01,
          3.9697e-01, -5.2783e-01,  1.

In [13]:
image_embedding[0]

tensor([ 3.4985e-01,  1.2634e-01, -1.5808e-01,  9.9182e-02,  1.6418e-01,
        -1.9592e-01,  6.2646e-01,  1.9360e-01, -1.2854e-01,  5.4626e-02,
        -4.6326e-02, -1.9714e-01,  1.0095e-01, -4.1748e-01,  3.0151e-01,
        -3.0176e-01, -1.8291e+00, -2.3361e-02,  2.1667e-01, -4.6851e-01,
         2.2705e-01,  1.1414e-01,  1.2927e-01,  2.0386e-01, -7.0190e-03,
        -1.7065e-01, -4.4312e-02, -2.9272e-01,  1.7822e-01,  1.2939e-01,
        -4.6265e-02,  5.4413e-02, -5.4053e-01, -6.1920e-02, -1.0248e-01,
         1.7175e-01, -3.1421e-01,  3.7659e-02,  1.4526e-01, -7.5000e-01,
        -6.3965e-01, -1.5271e-01, -2.6855e-01, -4.7559e-01, -1.4819e-01,
         8.5303e-01,  5.4053e-01, -2.0520e-01, -3.2666e-01,  3.0249e-01,
         5.1465e-01, -3.4351e-01,  1.3733e-01,  2.6660e-01,  2.0898e-01,
         6.0596e-01, -1.9250e-01,  2.2675e-02, -7.4158e-02,  1.6327e-03,
         1.2732e-01, -7.2693e-02,  4.0308e-01,  1.1841e-01,  1.5576e-01,
         3.9673e-01, -5.2686e-01,  1.0413e-01, -1.5

In [2]:
from audioldm.clap.encoders import CLAPAudioEmbeddingClassifierFreev2
CLAP = CLAPAudioEmbeddingClassifierFreev2(
    key='waveform',
    pretrained_path="/home/ubuntu/project/v2a-mapper/pretrain/clap_htsat_tiny.pt",
    sampling_rate=16000,
    embed_mode="audio",
    amodel="HTSAT-tiny"
)

DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/vocab.json HTTP/1.1" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/vocab.json HTTP/1.1" 200 0
INFO:root:Loading HTSAT-tiny model config.
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torchlibrosa/stft.py:193: FutureWarning: Pass size=1024 as keyword args. From version 0.10 passing these as positional arguments will result in an error
  fft_window = librosa.util.pad_center(fft_window, n_fft)
/home/ubuntu/miniconda3/envs/v2a-mapper/lib/python3.11/site-packages/torch/functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:3637.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /roberta-base/resolve/main/config.js

In [10]:
test_file = "/mnt/new_volume/vgg_sound/test.csv"

In [17]:
# change to column name
# df_test = pd.read_csv(test_file, columns=["path", "label"])
df_test = pd.read_csv(test_file, header=None)

In [18]:
df_test

,0,1
0,glLQrEijrKg_000300.mp4,playing hammond organ
1,LDoXsip0BEQ_000177.mp4,parrot talking
2,6jiO0tPLK7U_000090.mp4,basketball bounce
3,cRlp5v9BHeE_000011.mp4,car passing by
4,WgZ8KAnnTb8_000030.mp4,rapping
...,...,...
15441,ZPeG32vKQ3w_000030.mp4,typing on typewriter
15442,yyqydavJ4YQ_000201.mp4,airplane flyby
15443,EJwOhdjhxXY_000012.mp4,francolin calling
15444,oYEzy8gH6q8_000030.mp4,tapping guitar


In [19]:
vgg_dataset = VGGSound(root="/mnt/new_volume/vgg_sound/scratch/shared/beegfs/hchen/train_data/VGGSound_final/video",
                           csv_path="/mnt/new_volume/vgg_sound/test.csv",
                           sample_rate =  16000)


# data = vgg_dataset[7]

INFO:root:199176 videos found in /mnt/new_volume/vgg_sound/scratch/shared/beegfs/hchen/train_data/VGGSound_final/video
INFO:root:15446 videos found in /mnt/new_volume/vgg_sound/test.csv
INFO:root:0 videos missing in /mnt/new_volume/vgg_sound/scratch/shared/beegfs/hchen/train_data/VGGSound_final/video


In [22]:
a =vgg_dataset[0]['clip_video']

sample_rate: 44100
self sample:  16000


In [23]:
a.shape

torch.Size([64, 3, 224, 224])